In [ ]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

# Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import chardet
import folium as fm
import branca as br 

from folium import Marker, GeoJson, Choropleth
from folium.plugins import MarkerCluster, HeatMap, StripePattern

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString

In [ ]:
#first we read the file
shapefile_answer = gpd.read_file(r'C:\Users\analu\OneDrive\Escritorio\QLAB_Summer_Python\_data\INEI_LIMITE_DEPARTAMENTAL\INEI_LIMITE_DEPARTAMENTAL.shp')

print(shapefile_answer.columns) #then we explore its columns

#first we read the file

df4 = sex_work.groupby( ['dpt_code', 'month', 'sex'], as_index = False )[['empl']].sum() \
        .pivot( index = [ 'dpt_code', 'month' ] , columns = 'sex',values='empl') \
        .reset_index()
df4[ 'prop_wom' ] = ( df4.Mujer * 100 / df4.Hombre )

df5 = dpt_shp.merge(df4, left_on = 'CCDD', right_on = 'dpt_code') #now we merge this df 

# Inverting colour map
cmap = plt.cm.OrRd

fig, axis = plt.subplots( nrows = 4, ncols= 3, figsize = ( 15, 15 ) )

idx = 0
for i in range( 4 ):
    for j in range ( 3 ):
        
        
        ax = axis[ i ][ j ]
        
        month = df5.month.unique()[ idx ]
        
        df6 = df5[ df5.month == month ]
        
        df6.plot( column='prop_wom', 
                  cmap='Reds', 
                  linestyle='--',
                  edgecolor='black', 
                  legend = True, 
                  ax = ax 
                )
        
        ax.set_title( month )
        
        idx = idx + 1

from textwrap import wrap
# Inverting colour map
cmap = plt.cm.OrRd

fig, ax = plt.subplots(figsize=(20, 20))
df6.plot( ax = ax, 
        column='prop_wom', 
         cmap= cmap, 
         figsize=(20, 20), 
         linestyle='--',
         edgecolor='black', 
         legend = True,  
         scheme = "User_Defined", 
         classification_kwds = dict( bins = [ 20, 30, 40, 50,  100 ] ), 
         legend_kwds=dict(  loc='upper left',
                            bbox_to_anchor=(1.01, 1),
                            fontsize='x-large',
                            title= "Women Proportion", 
                            title_fontsize = 'x-large', 
                            frameon= False ))


In [ ]:
# First we Read the shapefile
shapefile_answer = gpd.read_file(r'C:\Users\analu\OneDrive\Escritorio\QLAB_Summer_Python\_data\INEI_LIMITE_DEPARTAMENTAL\INEI_LIMITE_DEPARTAMENTAL.shp')

# Then we Get aggregated data
df4 = sex_work.groupby(['dpt_code', 'month', 'sex'], as_index=False)[['empl']].sum() \
        .pivot(index=['dpt_code', 'month'], columns='sex', values='empl') \
        .reset_index()
df4['prop_wom'] = (df4.Mujer * 100 / df4.Hombre)

# Merge with the shapefile
df5 = shapefile_answer.merge(df4, left_on='CCDD', right_on='dpt_code')

# Now we create a folium map centered at a middle point, with a zoom of 6
map2 = folium.Map(location=[-9.19, -75.0152], zoom_start=6)

# Create subplots
fig, axis = plt.subplots(nrows=4, ncols=3, figsize=(15, 15))

# Create a loop for to forIterate over the subplots
idx = 0
for i in range(4):
    for j in range(3):
        # Get the current axis
        ax = axis[i][j]

        # Get the corresponding month
        month = df5.month.unique()[idx]

        # Filter the DataFrame by month
        df6 = df5[df5.month == month]

        # Create the choropleth map
        Choropleth(
            geo_data=df6,
            data=df6,
            columns=['geometry', 'prop_wom'],
            fill_color='OrRd',
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name=f'Women Proportion - {month}',
        ).add_to(map2)
        

        # Add polygon boundaries 
        folium.GeoJson(df6, name=f'geojson_{month}').add_to(map2)

        # Create the plot on the current axis
        df6.plot(
            column='prop_wom',
            cmap='Reds',
            linestyle='--',
            edgecolor='black',
            legend=True,
            ax=ax
        )

        # Set the title
        ax.set_title(month)

        idx += 1

# Add the folium map at the end
folium.LayerControl().add_to(map2)

#save the map 
map2.save("map2.html")

# Show the map
map2

# FIrst, we create a Folium map centered at the desired location
map3 = folium.Map(location=[-9.19, -75.0152], zoom_start=6)  

# Then, we create subplots
fig, ax = plt.subplots(figsize=(20, 20))

# This way we handle NaN values
df6.loc[(df6.NOMBDEP == 'LIMA'), 'prop_wom'] = np.nan

# Get the inverted color palette
cmap = plt.cm.OrRd_r

# Create the choropleth map
Choropleth(
    geo_data=df6,
    data=df6,
    columns=['geometry', 'prop_wom'],
    key_on='feature.properties.geometry',
    fill_color=cmap,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Women Proportion',
    bins=[20, 30, 40, 50, 100],
    missing_kwds=dict(color="#DADADB"),
).add_to(map3)

# Add polygon boundaries (optional)
folium.GeoJson(df6, name='geojson').add_to(map3)

# Create the plot on the current axis
df6.plot(
    column='prop_wom',
    cmap=cmap,
    linestyle='--',
    edgecolor='black',
    legend=True,
    ax=ax
)

# Configure legend
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=20, vmax=100))
sm._A = []
plt.colorbar(sm, cax=cax, label="Women Proportion")

# Configure title
ax.set_title("Women Proportion")

# Save the map as an HTML file
map3.save("map3.html")

# Show the map
map3